In [362]:
import pandas as pd

# Load CSV files
match_results_df = pd.read_csv('csv/wedstrijden_onverwerkd.csv')
goal_events_df = pd.read_csv('csv/doelpunten_onverwerkd.csv')
standings_df = pd.read_csv('csv/klassement_onverwerkd.csv')

goal_events_df = goal_events_df[goal_events_df['valid_goal'] == True]
# Filter of vervang niet-numerieke waarden
match_results_df['doelpunten_thuisploeg'] = pd.to_numeric(match_results_df['doelpunten_thuisploeg'], errors='coerce').fillna(0)
match_results_df['doelpunten_uitploeg'] = pd.to_numeric(match_results_df['doelpunten_uitploeg'], errors='coerce').fillna(0)

# Nu kunt u de conversie naar int veilig uitvoeren
match_results_df['doelpunten_thuisploeg'] = match_results_df['doelpunten_thuisploeg'].astype(int)
match_results_df['doelpunten_uitploeg'] = match_results_df['doelpunten_uitploeg'].astype(int)

In [363]:
# Vind alle verplaatste matches, namelijk de match id zal dezelfde zijn als de vorige
match_results_df['same_as_previous_match'] = match_results_df['id_match'].eq(match_results_df['id_match'].shift())

# Selecteer alleen de rijen waar 'same_as_previous_match' True is
duplicate_matches = match_results_df[match_results_df['same_as_previous_match']]
duplicate_matches.head(30)

,seizoen,speeldag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg_stamnummer,uitploeg,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match


In [364]:
# Bereken goals
goals_home_team = goal_events_df[goal_events_df['goal_team_stamnummer'] == goal_events_df['thuisploeg_stamnummer']].groupby('id_match').size()
goals_away_team = goal_events_df[goal_events_df['goal_team_stamnummer'] == goal_events_df['uitploeg_stamnummer']].groupby('id_match').size()

# Omzetting
goals_home_team_df = goals_home_team.reset_index(name='calculated_doelpunten_thuisploeg')
goals_away_team_df = goals_away_team.reset_index(name='calculated_doelpunten_uitploeg')

# Vervang NaN values in goal kolom met 0
merged_df = pd.merge(match_results_df, goals_home_team_df, on='id_match', how='outer')
merged_df = pd.merge(merged_df, goals_away_team_df, on='id_match', how='outer')
merged_df['calculated_doelpunten_thuisploeg'] = merged_df['calculated_doelpunten_thuisploeg'].fillna(0).astype(int)
merged_df['calculated_doelpunten_uitploeg'] = merged_df['calculated_doelpunten_uitploeg'].fillna(0).astype(int)

# Bereken verschil tussen berekening en resultaat
discrepancies_goals = merged_df[
    ((merged_df['calculated_doelpunten_thuisploeg'] != merged_df['doelpunten_thuisploeg']) |
    (merged_df['calculated_doelpunten_uitploeg'] != merged_df['doelpunten_uitploeg'])) &
    ~((merged_df['doelpunten_thuisploeg'] == 5) & (merged_df['doelpunten_uitploeg'] == 0)) &
    ~((merged_df['doelpunten_thuisploeg'] == 0) & (merged_df['doelpunten_uitploeg'] == 5))
]

# Select only relevant columns for display - Lommel 2002, Excelsior 2009 verwijderen we alles van, matches, klassement en doelpunten
discrepancies_goals.head(1000)

,seizoen,speeldag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg_stamnummer,uitploeg,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match,calculated_doelpunten_thuisploeg,calculated_doelpunten_uitploeg
12602,1979,2,1979/09/01,20:00:00,4013213,47,RWD Molenbeek,522,Beringen FC,1,0,False,0,0
12609,1979,2,1979/09/02,15:00:00,4013220,30,Lierse SK,553,Waterschei SV Thor,3,1,False,4,1
13329,1977,14,1977/11/20,15:00:00,4012716,19,KV Kortrijk,4451,KSV Waregem,1,1,False,0,0
13330,1977,15,1977/11/26,19:30:00,4012718,22,R Charleroi SC,35,RSC Anderlecht,2,1,False,0,0
13335,1977,15,1977/11/27,15:00:00,4012722,2300,KSK Beveren,4,RFC Luik,2,0,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15601,1969,13,1970/03/21,16:00:00,4010564,522,Beringen FC,4,RFC Luik,2,2,False,0,0
15602,1969,14,1969/12/05,20:00:00,4010458,10,Union Saint-Gilloise,373,Sint-Truidense VV,3,1,False,0,0
15603,1969,14,1969/12/07,14:30:00,4010459,55,RCC Schaerbeek,16,Standard Luik,3,1,False,0,0
15604,1969,14,1969/12/07,14:30:00,4010460,522,Beringen FC,35,RSC Anderlecht,2,2,False,0,0


In [365]:
#Controle voor (5:0 of 0:5's)
discrepancies_goals = merged_df[
    (merged_df['calculated_doelpunten_thuisploeg'] != merged_df['doelpunten_thuisploeg']) |
    (merged_df['calculated_doelpunten_uitploeg'] != merged_df['doelpunten_uitploeg'])
]
discrepancies_goals.head(1000)

,seizoen,speeldag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg_stamnummer,uitploeg,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match,calculated_doelpunten_thuisploeg,calculated_doelpunten_uitploeg
356,2022,14,2022/10/23,18:30:00,3851182,16,Standard Luik,35,RSC Anderlecht,5,0,False,3,1
380,2022,17,2022/11/12,20:45:00,3851244,22,R Charleroi SC,25,KV Mechelen,0,5,False,1,0
728,2021,22,2022/01/15,16:15:00,3598295,18,Oud-Heverlee Leuven,25,KV Mechelen,5,0,False,0,0
841,2021,34,2022/04/10,18:30:00,3598351,10,Union Saint-Gilloise,13,Beerschot,5,0,False,0,0
2427,2014,11,2014/10/19,14:30:00,2503727,16,Standard Luik,5381,SV Zulte Waregem,0,5,False,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15564,1969,9,1969/11/02,15:00:00,4010423,35,RSC Anderlecht,13,Beerschot,0,1,False,0,0
15566,1969,9,1969/11/02,15:00:00,4010425,22,R Charleroi SC,3,Club Brugge,3,1,False,0,0
15567,1969,9,1969/11/02,15:00:00,4010426,53,AS Oostende KM,55,RCC Schaerbeek,2,0,False,0,0
15568,1969,9,1969/11/02,15:00:00,4010427,7,KAA Gent,373,Sint-Truidense VV,4,1,False,0,0


In [366]:
# controle van aantal_wedstrijden
# Geen enkel record met meer wedstrijden dan speeldagen
number_of_games_check = standings_df[standings_df.speeldag < standings_df.aantal_wedstrijden]
number_of_games_check.head(30)

,seizoen,speeldag,stand,naam_ploeg,id_ploeg,aantal_wedstrijden,aantal_gewonnen,aantal_gelijk,aantal_verloren,doelpunten_voor,doelpunten_tegen,verschil,punten


In [367]:
# controle van aantal_wedstrijden
# wel records met minder wedstrijden dan speeldagen, maar bij nazicht van een aantal records klopt dit
number_of_games_check = standings_df[standings_df.speeldag > standings_df.aantal_wedstrijden]
number_of_games_check.seizoen.value_counts()

1975    551
2002    442
2009    360
Name: seizoen, dtype: int64

In [368]:
# geen records met standings_df.Played != (standings_df.Wins + standings_df.Losses + standings_df.Draws
number_of_games_check = standings_df[standings_df.aantal_wedstrijden != (standings_df.aantal_gewonnen + standings_df.aantal_gelijk + standings_df.aantal_verloren)]
number_of_games_check.count()

seizoen               0
speeldag              0
stand                 0
naam_ploeg            0
id_ploeg              0
aantal_wedstrijden    0
aantal_gewonnen       0
aantal_gelijk         0
aantal_verloren       0
doelpunten_voor       0
doelpunten_tegen      0
verschil              0
punten                0
dtype: int64

In [369]:
# geen records met (uitslagen.doelpunten_voor - uitslagen.doelpunten_tegen) != (uitslagen.verschil)
standings_df['calculated_goal_verschil'] = standings_df['doelpunten_voor'] - standings_df['doelpunten_tegen']
controle_aantal_wedstrijden = standings_df[standings_df['calculated_goal_verschil'] != standings_df['verschil']]
controle_aantal_wedstrijden.count()

seizoen                     0
speeldag                    0
stand                       0
naam_ploeg                  0
id_ploeg                    0
aantal_wedstrijden          0
aantal_gewonnen             0
aantal_gelijk               0
aantal_verloren             0
doelpunten_voor             0
doelpunten_tegen            0
verschil                    0
punten                      0
calculated_goal_verschil    0
dtype: int64

In [370]:
# het totaal aantal gewonnen matchen per seizoen en per ploeg
points_df = standings_df.groupby(['seizoen','speeldag','id_ploeg', 'punten'])[['aantal_gewonnen','aantal_gelijk','aantal_verloren']].sum()
points_df = points_df.reset_index()
points_df = points_df.sort_values(['seizoen','speeldag','aantal_gewonnen','aantal_gelijk','aantal_verloren'], ascending = [True, True, False, False, False])

In [371]:
points_df['punten_check'] = points_df['aantal_gewonnen'] * 3 + points_df['aantal_gelijk'] + points_df['aantal_verloren'] * 0
discrepancies_df = points_df[points_df['punten'] != points_df['punten_check']]
discrepancies_df.head(100)

,seizoen,speeldag,id_ploeg,punten,aantal_gewonnen,aantal_gelijk,aantal_verloren,punten_check
1,1960,1,2,2:0,1,0,0,3
3,1960,1,4,2:0,1,0,0,3
5,1960,1,10,2:0,1,0,0,3
6,1960,1,13,2:0,1,0,0,3
7,1960,1,16,2:0,1,0,0,3
...,...,...,...,...,...,...,...,...
95,1960,6,3434,2:10,0,2,4,2
103,1960,7,16,12:2,5,2,0,17
97,1960,7,2,10:4,4,2,1,14
99,1960,7,4,10:4,4,2,1,14


In [372]:
match_results_df['datum'] = pd.to_datetime(match_results_df['datum'])
goal_events_df['datum'] = pd.to_datetime(goal_events_df['datum'])

merged_df = pd.merge(goal_events_df, match_results_df, on='id_match', suffixes=('_goal', '_match'))

# Check for discrepancies in dates
date_discrepancies_df = merged_df[merged_df['datum_goal'] != merged_df['datum_match']]
date_discrepancies_df.head(30)

,seizoen_goal,dag,datum_goal,tijd_goal,id_match,thuisploeg_stamnummer_goal,thuisploeg_goal,uitploeg_goal,uitploeg_stamnummer_goal,goal_time,...,speeldag,datum_match,tijd_match,thuisploeg_stamnummer_match,thuisploeg_match,uitploeg_stamnummer_match,uitploeg_match,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match


In [373]:
# Groepeer de gegevens op 'Day' en controleer de datums
sorted_dates = match_results_df.sort_values(by=['speeldag', 'datum'])
date_discrepancies_df = sorted_dates.groupby('speeldag')['datum'].apply(lambda x: x.is_monotonic_increasing)

date_discrepancies_df = date_discrepancies_df[date_discrepancies_df == False]
date_discrepancies_df.head(30)


Series([], Name: datum, dtype: bool)

In [374]:
# Sorting the data by season, day, and date
sorted_dates = match_results_df.sort_values(by=['seizoen', 'speeldag', 'datum'])

# Group by 'season' and 'day', and check if dates are in chronological order within each group
grouped = sorted_dates.groupby(['seizoen', 'speeldag'])
date_order_check = grouped['datum'].apply(lambda x: x.is_monotonic_increasing)

# Identifying seasons and days where dates are not in chronological order
date_discrepancies = date_order_check[date_order_check == False]
date_discrepancies.head(30)

Series([], Name: datum, dtype: bool)

In [375]:
# Filter matches that ended 0-0
matches_0_0 = match_results_df[(match_results_df['doelpunten_thuisploeg'] == 0) & (match_results_df['doelpunten_uitploeg'] == 0)]

# Get match_ids of 0-0 matches
match_ids_0_0 = matches_0_0['id_match']

# Check if these matches have any goals in goal_events.csv
goals_in_0_0_matches = goal_events_df[goal_events_df['id_match'].isin(match_ids_0_0)]
goals_in_0_0_matches.head(100)

,seizoen,dag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg,uitploeg_stamnummer,goal_time,real_time_goal,goal_team_naam,goal_team_stamnummer,stand_thuis,stand_uit,valid_goal


In [376]:
# Sorteer de DataFrame op basis van seizoen, speeldag, punten, aantal gewonnen wedstrijden, doelpuntenverschil en gemaakte doelpunten
sorted_df = standings_df.sort_values(['seizoen', 'speeldag', 'punten', 'aantal_gewonnen', 'calculated_goal_verschil', 'doelpunten_voor'], ascending=[True, True, False, False, False, False])

# Bereken de stand
sorted_df['calculated_stand'] = sorted_df.groupby(['seizoen', 'speeldag']).cumcount() + 1

# Pas de stand aan voor gelijke standen
sorted_df['calculated_stand'] = sorted_df.groupby(['seizoen', 'speeldag', 'punten', 'aantal_gewonnen', 'calculated_goal_verschil', 'doelpunten_voor'])['calculated_stand'].transform('min')

# Voeg de berekende stand toe aan de originele DataFrame
standings_df = standings_df.merge(sorted_df[['seizoen', 'speeldag', 'id_ploeg', 'calculated_stand']], on=['seizoen', 'speeldag', 'id_ploeg'], how='left')

# Controleer op discrepanties tussen de berekende en werkelijke stand
discrepancies = standings_df[standings_df['stand'] != standings_df['calculated_stand']]
discrepancies.head(1000)


,seizoen,speeldag,stand,naam_ploeg,id_ploeg,aantal_wedstrijden,aantal_gewonnen,aantal_gelijk,aantal_verloren,doelpunten_voor,doelpunten_tegen,verschil,punten,calculated_goal_verschil,calculated_stand
48,2023,4,1,Club Brugge,3,4,3,1,0,14,2,12,10,12,12
49,2023,4,2,KAA Gent,7,4,3,1,0,9,5,4,10,4,13
50,2023,4,3,Union SG,10,4,3,0,1,8,5,3,9,3,1
51,2023,4,4,RSC Anderlecht,35,4,3,0,1,4,3,1,9,1,2
52,2023,4,5,Antwerp FC,1,4,2,1,1,8,2,6,7,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2478,2019,12,3,Antwerp FC,1,12,7,2,3,26,17,9,23,9,5
2479,2019,12,4,KV Mechelen,25,12,7,2,3,22,18,4,23,4,6
2480,2019,12,5,KAA Gent,7,12,6,3,3,24,15,9,21,9,7
2481,2019,12,6,KRC Genk,322,12,6,2,4,19,15,4,20,4,8


In [377]:
filtered_matches = match_results_df[(match_results_df['doelpunten_thuisploeg'] == 5) & (match_results_df['doelpunten_uitploeg'] == 0) | (match_results_df['doelpunten_thuisploeg'] == 0) & (match_results_df['doelpunten_uitploeg'] == 5)]

filtered_matches.head(100)

,seizoen,speeldag,datum,tijd,id_match,thuisploeg_stamnummer,thuisploeg,uitploeg_stamnummer,uitploeg,doelpunten_thuisploeg,doelpunten_uitploeg,same_as_previous_match
21,2023,3,2023-08-13,16:00:00,4098800,4276,KAS Eupen,3,Club Brugge,0,5,False
356,2022,14,2022-10-23,18:30:00,3851182,16,Standard Luik,35,RSC Anderlecht,5,0,False
376,2022,17,2022-11-11,20:45:00,3851239,18,Oud-Heverlee Leuven,17,RFC Seraing,5,0,False
380,2022,17,2022-11-12,20:45:00,3851244,22,R Charleroi SC,25,KV Mechelen,0,5,False
481,2022,28,2023-03-05,16:00:00,3851121,1,Royal Antwerp FC,25,KV Mechelen,5,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
7650,1996,29,1997-04-05,20:00:00,3183694,90,Eendracht Aalst,3530,Germinal,5,0,False
7820,1995,14,1995-10-28,20:00:00,3491975,90,Eendracht Aalst,373,Sint-Truidense VV,0,5,False
7822,1995,15,1995-11-03,20:00:00,3491977,17,RFC Seraing,90,Eendracht Aalst,5,0,False
7879,1995,21,1996-01-27,20:00:00,3492034,373,Sint-Truidense VV,35,RSC Anderlecht,0,5,False
